In [24]:
import numpy as np
import jax
import jax.numpy as jnp
import scipy.stats as ss
import tensorly as tl
tl.set_backend('jax')
# import matplotlib.pyplot as plt
# %matplotlib inline
# from IPython.display import display
# import sympy; sympy.init_printing()

# def display_matrix(m):
#     display(sympy.Matrix(m))

In [40]:
sample = np.genfromtxt('references/sample.csv', delimiter=',')

In [46]:
S = jnp.transpose(sample)
print(S[:3,:3])

[[1.05       1.04856412 1.01787965]
 [1.05       1.02942462 1.00856132]
 [1.05       1.12692091 1.11423357]]


In [47]:
# check distribution
print(sample.shape)
# print(S[:,-1][:5])

print(np.mean(S[:,-1]))
print(np.var(S[:,-1]))

# 1.05

(100, 10000)
1.0518324048639505
0.07034371653276578


In [51]:
N = S.shape[0]          # number of time steps
r = 0.05       # interest rate
K = 1          # strike 
T = 1          # Maturity

dt = T/(N-1)          # time interval
df = jnp.exp(-r * dt)  # discount factor per time interval

In [52]:
H = np.maximum(K - S, 0)           # intrinsic values for put option

V = np.zeros_like(H)               # value matrix
V[:,-1] = H[:,-1]

# Valuation by LS Method
for t in range(N-2, 0, -1):

    good_paths = H[:,t] > 0        # paths where the intrinsic value is positive 
                                   # the regression is performed only on these paths 
    
    rg = jnp.polyfit( S[good_paths, t], V[good_paths, t+1] * df, 2)    # polynomial regression
    C = jnp.polyval( rg, S[good_paths,t] )                             # evaluation of regression  
    
    exercise = np.zeros( len(good_paths), dtype=bool)    # initialize
    exercise[good_paths] = H[good_paths,t] > C           # paths where it is optimal to exercise
    
    V[exercise,t] = H[exercise,t]                        # set V equal to H where it is optimal to exercise 
    V[exercise,t+1:] = 0                                 # set future cash flows, for that path, equal to zero  
    discount_path = (V[:,t] == 0)                        # paths where we didn't exercise 
    V[discount_path,t] = V[discount_path,t+1] * df       # set V[t] in continuation region
    
V0 = jnp.mean(V[:,1]) * df  # discounted expectation of V[t=1]
print("Example price= ", V0)

IndexError: index 9998 is out of bounds for axis 1 with size 100

In [30]:
print(H.round(2))

[[0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.41 0.44 0.43]
 [0.   0.   0.   ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]]


In [31]:
# 0.0797